# Calculating similarity using MinHash and MapReduce

Now we are going to reuse the code to download Wikipedia articles, and then we will use MinHash to estimate the Jaccard similarity between those articles. To do things properly, we are going to tokenise and lemmatise the articles using NLTK (an Natural Processing Language library).

We will develop this as a MapReduce task.




In [1]:
import concurrent.futures
import re

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.stem import SnowballStemmer

import requests

ModuleNotFoundError: No module named 'nltk'

In [ ]:
def get_wikipedia_article(wiki_article, timeout=10):
    '''
    Method that gets a Wikipedia page. 
    '''
    wiki_page_url = 'https://en.wikipedia.org/wiki/'+wiki_article
    response = requests.get(url=wiki_page_url, timeout=timeout)
    article = response.text
    return article


def get_wikipedia_articles(wiki_articles):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for article in wiki_articles:
            futures.append(executor.submit(get_wikipedia_article,
                                           wiki_article=article)
                          )
        articles = []
        for future in concurrent.futures.as_completed(futures):
            articles.append(future.result())
    return articles


# Let's define some functions to clean HTML code, punctuation marks and symbols from the original articles
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

def clean_punctuation_marks(text):
    return re.sub(r'[^\w\s]',' ', text)

def clean_return_and_tabs(text):
    return re.sub(r'[\n, \t]',' ', text)

def clean_article(data):
    cdata = clean_html(data)
    cdata = clean_punctuation_marks(cdata)
    cdata = clean_return_and_tabs(cdata)
    return cdata

### Now it is time to fetch some concepts

We will download the Wikipedia articles specified in the list below. After you run the simulation for the first time, feel free to modify them to test with other terms!

We will run the simulation with 'lion', 'beetle' and 'tiger'. Which ones do you think should have a greater similarity? Well both lions and tigers are large felines while the beetle is an insect...

In [ ]:
wiki_articles = [
    'lion',
    'beetle',
    'tiger',
]

# Other examples: 
# (I am intentionally putting a few concepts that should be related between them and one that is not, 
# but feel free to put whatever you want in there, as long as there is a Wikipedia article for it)

# wiki_articles = ['Ford', 'Audi', 'Mercedes-Benz', 'lamb']
# wiki_articles = ['Donald_Trump', 'Barack_Obama', 'computer', 'Bill_Clinton']

articles = get_wikipedia_articles(wiki_articles)

#### Now that we have the articles, let's do some basic cleaning and NLP on them:

Basic cleaning -> tokenise -> remove stopwords -> stem -> remove numbers -> remove specific stopwords

In [ ]:
tokeniser = tokenize.NLTKWordTokenizer()
stemmer = SnowballStemmer('english')

In [ ]:
# First we make it all lowercase and do a basic cleaning as we did in previous interactive activities
articles = map(lambda a: a.lower(), articles)
articles = map(clean_article, articles)
# Then we tokenise the articles, extracing the words from them
articles = map(tokeniser.tokenize, articles)

# We will also remove the English stopwords 
# (you can check which ones are the English stopwords by checking this: print(stopwords.words('english')))

stopwords = set(['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 
                'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 
                'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 
                'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 
                'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 
                'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 
                'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 
                'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 
                'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 
                'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 
                'doing', 'it', 'how', 'further', 'was', 'here', 'than'])

def remove_stopwords(document, stopwords):
    output_doc = []
    for word in document:
        if word not in stopwords:
            output_doc.append(word)
    return output_doc

articles = map(lambda a: remove_stopwords(a, stopwords), articles)

# And then we stem those words to remove plurals and other variations
articles = [map(stemmer.stem, article) for article in articles]


def remove_numbers(document):
    numbers = set(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    output_doc = []
    for word in document:
        if not any(num in word for num in numbers):
            output_doc.append(word)
    return output_doc

articles = map(remove_numbers, articles)

def remove_wiki_and_wg_words(document):
    output_doc = []
    for word in document:
        if not word.startswith('wg') and not word.startswith('wiki'):
            output_doc.append(word)
    return output_doc

articles = map(remove_wiki_and_wg_words, articles)
articles = list(articles)

### Real Jaccard similarity:

We can now use those cleaned articles to calculate their real Jaccard similarities. We can do this because we are just dealing with a few articles and they are relatively short documents. If we had to do this for all websites on the internet, or even just a few tens of thousands of articles, it wouldn't be feasible as it is.

You should see that the Tiger and Lion articles should be more similar between them than with the Beetle one!

In [ ]:
def jaccard(a1, a2):
    set1 = set(a1)
    set2 = set(a2)
    union_size = len(set1.union(set2))
    intersection_size = len(set1.intersection(set2))
    return round(intersection_size/union_size, 4)

for i in range(len(articles)-1):
    for j in range(i+1, len(articles)):
        print('Similarity between', articles[i][0], 'and', articles[j][0], ':',jaccard(articles[i], articles[j]))

### MinHash:

Now let's estimate those real Jaccard measures with the MinHash algorithm we saw in the session and see how good our results are...

First we will create 3-word shingles (3-grams) and hash them for all of the documents

In [ ]:
import binascii  # To transform (hash) each shingle to a 32-bit integer 
import random

In [ ]:
def shingle_article(article, shingle_size=3):
    hashed_shingles = [] # Shingles are those 3 words transformed in an integer using CRC32 algorithm
    for i in range(len(article)-(shingle_size-1)):
        # We put groups (shingles) of 3 words together, separated by a space
        # We could change the number of words per shingle, changing shingle_size :)
        shingle = ' '.join([article[i+j] for j in range(shingle_size)])
        crc_hash = binascii.crc32(bytes(shingle.encode('UTF-8')))
        hashed_shingles.append(crc_hash)
    return hashed_shingles

SHINGLE_SIZE = 2

# We "save" the first word of the article in order to be able to idenfity it later when we hash it all 
shingled_articles = map(lambda a: (a[0], shingle_article(a, shingle_size=SHINGLE_SIZE)), articles)
shingled_articles = list(shingled_articles)

In [ ]:
maxCoefficient = 2**32-1  # Because we are transforming shingles into 32-bit integers, we know their maximum
big_prime = 4294967311  # And we found their next primer in internet (when you work with 32-bit ints take that number)


N = 20 # Using 20 hashing functions, feel free to modify!

# We will use N hashing functions, so we need to randomly take N As and N Bs for our N hashing functions
# Our hashing functions will be of the form: hashed_value = A * value + B % big_prime
a_coeffs = [random.randint(0, maxCoefficient) for i in range(N)]
b_coeffs = [random.randint(0, maxCoefficient) for i in range(N)]

In [ ]:
def get_hashed_value(shingle, a_coeff, b_coeff, big_prime):
    return ((shingle * a_coeff) + b_coeff) % big_prime

# For each article, and each of the N hash functions, we calculate the hashes of all the shingles
# and get the minimum of those hashes. This will create a signature of size N for each document. 
# Finally, to estimate their Jaccard similarity, we just only compare the signatures
article_signatures = {}
article_names = []
for article_name, shingled_article in shingled_articles:
    article_signature = []
    for hashing_function_index in range(N):
        a_coeff = a_coeffs[hashing_function_index]
        b_coeff = b_coeffs[hashing_function_index]
        hashes = map(lambda shingles: get_hashed_value(shingles, a_coeff, b_coeff, big_prime), shingled_article)
        article_signature.append(min(list(hashes)))
    article_signatures[article_name] = article_signature
    article_names.append(article_name)
    
for s in article_signatures:
    print('Signature for', s,': ',article_signatures[s])

#### Article signatures:

You can have a look at article_signatures just printed above to see what we are actually using to compare the articles. We have reduced documents of potentially thousands of words to just N numbers. 

We will apply Jaccard on those N numbers, and still lion and tiger should be more similar than lion and bettle or tiger and beetle.

In the code cells above, feel free to modify the number of hashes we are using, as well as the shingle_size (how many words per shingle).


* What happens if you put a large shingle size? (for example 20 words). Why do you think that is?
* And when we increase/decrease the number of hashing functions? Is the more always the better?
* Can you create a situation where the Beetle article is more similar to lion or tiger than between them?

In [ ]:
print('Using', N, 'hashing functions, and shingles of size',SHINGLE_SIZE, ', MinHas produces the following results:')
for i in range(len(article_names)-1):
    for j in range(i+1, len(article_names)):
        print('Similarity between', article_names[i], 'and', article_names[j], ':',
              jaccard(article_signatures[article_names[i]], article_signatures[article_names[j]]))

## Learning Exercises

This week's exercises are just to continue playing with the simulations. For example:

* Can you explain why MinHash is a good algorithm for detecting plagiarism? How would you set the parameters of (1) shingling size and (2) number of hash functions to tune it for plagiarism detection?